In [1]:
import tensorflow as tf
import numpy as np

In [2]:
BATCH_SIZE = 64

In [5]:
input_ph = tf.placeholder(tf.float32, (BATCH_SIZE, 6 * 20), 'input')
gt_ph = tf.placeholder(tf.float32, (BATCH_SIZE, 6))

In [4]:
out = tf.layers.dense(input_ph, 512, activation=tf.nn.relu)
out = tf.layers.dense(out, 512, activation=tf.nn.relu)
out = tf.layers.dense(out, 256, activation=tf.nn.relu)
out = tf.layers.dense(out, 6, activation=tf.nn.relu)

In [6]:
loss = tf.nn.l2_loss(out - gt_ph)

In [7]:
optimizer = tf.train.MomentumOptimizer(learning_rate=1e-3, momentum=0.9, use_nesterov=True)

In [8]:
opt = optimizer.minimize(loss)

In [9]:
# load data
# all_idx = np.arange(data.shape[0])

In [10]:
sess = tf.InteractiveSession()

In [11]:
# sess.run(tf.global_variables_initializer())

In [ ]:
for i in range(10000):
    idx = np.random.choice(data.shape[0])
    x, y = data_x[idx], data_y[idx]
    sess.run(opt, feed_dict={input_ph: x, gt_ph_y})
    if i % 100 == 0:
        sess.run(loss, feed_dict={input_ph: x, gt_ph_y})